# LanceDB based Multimodal Myntra Fashion Search Engine

## Preliminaries

In [ ]:
%%capture

!pip install lancedb
!pip install open_clip_torch

In [ ]:
import os
import argparse
import pandas as pd
from PIL import Image
from pathlib import Path
from random import sample

import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import EmbeddingFunctionRegistry

from typing import Any

## Download Data
- For this project you need to download the [Myntra Fashion Product Dataset]( https://www.kaggle.com/datasets/hiteshsuthar101/myntra-fashion-product-dataset) from Kaggle and store it here in the `input` folder.
- Note that while creating the table you shall pass the path of the folder in which the images are present, example `/content/input/images`

## Embedding Model

In [ ]:
def register_model(model_name: str) -> Any:
    """
    Register a model with the given name using LanceDB's EmbeddingFunctionRegistry.

    Args:
        model_name (str): The name of the model to register.

    Returns:
        model: The registered model instance.

    Usage:
    >>> model = register_model("open-clip")
    """
    registry = EmbeddingFunctionRegistry.get_instance()
    model = registry.get(model_name).create()
    return model

## Schema

In [ ]:
# Register the OpenAI CLIP model
clip = register_model("open-clip")


class Myntra(LanceModel):
    """
    Represents a Myntra Schema.

    Attributes:
        vector (Vector): The vector representation of the item.
        image_uri (str): The URI of the item's image.
    """

    vector: Vector(clip.ndims()) = clip.VectorField()
    image_uri: str = clip.SourceField()

    @property
    def image(self):
        return Image.open(self.image_uri)


# Function to map schema name to schema class
def get_schema_by_name(schema_name: str) -> Any:
    """
    Retrieves the schema object based on the given schema name.

    Args:
        schema_name (str): The name of the schema.

    Returns:
        object: The schema object corresponding to the given schema name, or None if not found.

    Usage:
    >>> schema = get_schema_by_name("Myntra")
    """
    schema_map = {
        "Myntra": Myntra,
    }
    return schema_map.get(schema_name)

## Creating a Table

In [ ]:
def create_table(
    database: str,
    table_name: str,
    data_path: str,
    schema: Any = Myntra,
    mode: str = "overwrite",
) -> None:
    """
    Create a table in the specified vector database and add data to it.

    Args:
        database (str): The name of the database to connect to.
        table_name (str): The name of the table to create.
        data_path (str): The path to the data directory.
        schema (Schema, optional): The schema to use for the table. Defaults to Myntra.
        mode (str, optional): The mode for creating the table. Defaults to "overwrite".

    Returns:
        None

    Usage:
    >>> create_table(database="~/.lancedb"", table_name="myntra", data_path="input")
    """

    # Connect to the lancedb database
    db = lancedb.connect(database)

    # Check if the table already exists in the database
    if table_name in db:
        print(f"Table {table_name} already exists in the database")
        table = db[table_name]

    # if it does not exist then create a new table
    else:

        print(f"Creating table {table_name} in the database")

        # Create the table with the given schema
        table = db.create_table(table_name, schema=schema, mode=mode)

        # Define the Path of the images and obtain the Image uri
        p = Path(data_path).expanduser()
        uris = [str(f) for f in p.glob("*.jpg")]
        print(f"Found {len(uris)} images in {p}")

        # Sample 1000 images from the data
        # Increase this value for more accurate results but
        # it will take more time to process embeddings
        uris = sample(uris, 100)

        # Add the data to the table
        print(f"Adding {len(uris)} images to the table")
        table.add(pd.DataFrame({"image_uri": uris}))
        print(f"Added {len(uris)} images to the table")

In [ ]:
# The data_path should refer to the folder in which the images are located

create_table(database="~/.lancedb", table_name="myntra", data_path="/content/input/images")

Creating table myntra_v4 in the database
Found 205 images in /content/input/images
Adding 100 images to the table


100%|██████████| 36/36 [00:09<00:00,  3.76it/s]

Added 100 images to the table


## Vector Search

In [ ]:
def run_vector_search(
    database: str,
    table_name: str,
    schema: Any,
    search_query: Any,
    limit: int = 6,
    output_folder: str = "output",
) -> None:
    """
    This function performs a vector search on the specified database and table using the provided search query.
    The search can be performed on either text or image data. The function retrieves the top 'limit' number of results
    and saves the corresponding images in the 'output_folder' directory. The function assumes if the search query ends
    with '.jpg' or '.png', it is an image search, otherwise it is a text search.
    Args:
        database (str): The path to the database.
        table_name (str): The name of the table.
        schema (Schema): The schema to use for converting search results to Pydantic models.
        search_query (Any): The search query, can be text or image.
        limit (int, optional): The maximum number of results to return. Defaults to 6.
        output_folder (str, optional): The folder to save the output images. Defaults to "output".

    Returns:
        None

    Usage:
    >>> run_vector_search(database="~/.lancedb", table_name="myntra", schema=Myntra, search_query="Black Kurta")

    """

    # Create the output folder if it does not exist
    if os.path.exists(output_folder):
        for file in os.listdir(output_folder):
            os.remove(os.path.join(output_folder, file))
    else:
        os.makedirs(output_folder)

    # Connect to the lancedb database
    db = lancedb.connect(database)

    # Open the table
    table = db.open_table(table_name)

    # Check if the search query is an image or text
    try:
        if search_query.endswith(".jpg") or search_query.endswith(".png"):
            search_query = Image.open(search_query)
        else:
            search_query = search_query
    except AttributeError as e:
        if str(e) == "'JpegImageFile' object has no attribute 'endswith'":
            print("Running via Streamlit, search query is already an array so skipping opening image using Pillow")
        else:
            raise

    # Perform the vector search and retrieve the results
    rs = table.search(search_query).limit(limit).to_pydantic(schema)

    # Save the images to the output folder
    for i in range(limit):
        image_path = os.path.join(output_folder, f"image_{i}.jpg")
        rs[i].image.save(image_path, "JPEG")

After the search is done, the results will be saved in the `output` folder.

## Text Search

In [ ]:
run_vector_search(database="~/.lancedb", table_name="myntra", schema=Myntra, search_query="White Kurta", limit=3, output_folder="output")

## Image Search

In [ ]:
run_vector_search(database="~/.lancedb", table_name="myntra", schema=Myntra, search_query="/content/input/images/0.jpg", limit=3, output_folder="output")